**This notebook is an exercise in the [Advanced SQL](https://www.kaggle.com/learn/advanced-sql) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/analytic-functions).**

---


# Introduction

Here, you'll use window functions to answer questions about the [Chicago Taxi Trips](https://www.kaggle.com/chicago/chicago-taxi-trips-bq) dataset.

Before you get started, run the code cell below to set everything up.

In [1]:
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql_advanced.ex2 import *
print("Setup Complete")

Using Kaggle's public dataset BigQuery integration.


/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


Setup Complete


The following code cell fetches the `taxi_trips` table from the `chicago_taxi_trips` dataset. We also preview the first five rows of the table.  You'll use the table to answer the questions below.

In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "chicago_taxi_trips" dataset
dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "taxi_trips" table
table_ref = dataset_ref.table("taxi_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.


,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,839771e41537432c54c7099e8ea1bf43887ca484,92195ba6d79380aa9da1f2db98ac05a3eff859c4151f24...,2015-05-01 18:00:00+00:00,2015-05-01 18:00:00+00:00,0,0.0,NaN,NaN,NaN,NaN,...,0.0,11.82,Credit Card,Chicago Elite Cab Corp. (Chicago Carriag,NaN,NaN,None,NaN,NaN,None
1,9437e8feb58e5cd3cb22a9bb91cbcf3bc60ee9a4,3e0ecc411309f519aa2278ee885227cde56891acadc652...,2015-05-03 11:15:00+00:00,2015-05-03 11:15:00+00:00,0,0.0,NaN,NaN,NaN,NaN,...,0.0,13.85,Credit Card,Suburban Dispatch LLC,NaN,NaN,None,NaN,NaN,None
2,8162b1f82324cf1270b4d1d7a5498ea5150501cf,36dee7e88e2048dfeb8a98c9e420a1600544a8d4f21fe6...,2015-05-09 01:15:00+00:00,2015-05-09 01:15:00+00:00,0,0.0,NaN,NaN,NaN,NaN,...,0.0,20.22,Credit Card,Chicago Elite Cab Corp. (Chicago Carriag,NaN,NaN,None,NaN,NaN,None
3,13f8cff0ef83a0e6feed5f67193e5b7e705eb7ab,35f259e52dda4ae7ef17bbcf2f61be54445372dc471929...,2015-04-14 14:45:00+00:00,2015-04-14 14:45:00+00:00,0,0.0,NaN,NaN,NaN,NaN,...,0.0,6.85,Credit Card,Chicago Elite Cab Corp. (Chicago Carriag,NaN,NaN,None,NaN,NaN,None
4,f4586d5fba35a944615fc6e75fc7c57a69cf9fa8,5b54ed6a93f455a3a35cec10d5708caa67ac3dfaf6727a...,2015-08-26 11:00:00+00:00,2015-08-26 11:15:00+00:00,540,1.7,NaN,NaN,NaN,NaN,...,0.0,7.45,Cash,None,NaN,NaN,None,NaN,NaN,None


# Exercises

### 1) How can you predict the demand for taxis?

Say you work for a taxi company, and you're interested in predicting the demand for taxis.  Towards this goal, you'd like to create a plot that shows a rolling average of the daily number of taxi trips.  Amend the (partial) query below to return a DataFrame with two columns:
- `trip_date` - contains one entry for each date from January 1, 2016, to December 31, 2017.
- `avg_num_trips` - shows the average number of daily trips, calculated over a window including the value for the current date, along with the values for the preceding 15 days and the following 15 days, as long as the days fit within the two-year time frame.  For instance, when calculating the value in this column for January 5, 2016, the window will include the number of trips for the preceding 4 days, the current date, and the following 15 days.

This query is partially completed for you, and you need only write the part that calculates the `avg_num_trips` column.  Note that this query uses a common table expression (CTE); if you need to review how to use CTEs, you're encouraged to check out [this tutorial](https://www.kaggle.com/dansbecker/as-with) in the [Intro to SQL](https://www.kaggle.com/learn/intro-to-sql) micro-course.

In [3]:
# Fill in the blank below
avg_num_trips_query = """
                      WITH trips_by_day AS
                      (
                          SELECT DATE(trip_start_timestamp) AS trip_date,
                              COUNT(*) as num_trips
                          FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                          WHERE trip_start_timestamp >= '2016-01-01' AND trip_start_timestamp < '2018-01-01'
                          GROUP BY trip_date
                          ORDER BY trip_date
                      )
                      SELECT trip_date,
                          AVG(num_trips)
                          OVER (
                              ORDER BY trip_date
                              ROWS BETWEEN 15 PRECEDING AND 15 FOLLOWING
                          ) AS avg_num_trips
                      FROM trips_by_day
                      """

# Check your answer
q_1.check()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,trip_date,avg_num_trips
0,2017-10-11,70399.806452
1,2017-10-10,70015.967742
2,2017-10-27,69811.290323
3,2016-07-31,89104.580645
4,2017-08-07,67715.806452


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [4]:
# Lines below will give you a hint or solution code
#q_1.hint()
#q_1.solution()

### 2) Can you separate and order trips by community area?

The query below returns a DataFrame with three columns from the table: `pickup_community_area`, `trip_start_timestamp`, and `trip_end_timestamp`.  

Amend the query to return an additional column called `trip_number` which shows the order in which the trips were taken from their respective community areas.  So, the first trip of the day originating from community area 1 should receive a value of 1; the second trip of the day from the same area should receive a value of 2.  Likewise, the first trip of the day from community area 2 should receive a value of 1, and so on.

Note that there are many numbering functions that can be used to solve this problem (depending on how you want to deal with trips that started at the same time from the same community area); to answer this question, please use the **RANK()** function.

In [6]:
# Amend the query below
trip_number_query = """
                    SELECT pickup_community_area,
                        trip_start_timestamp,
                        trip_end_timestamp,
                        RANK()
                            OVER (
                                PARTITION BY pickup_community_area
                                ORDER BY trip_start_timestamp
                            ) AS trip_number
                    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                    WHERE DATE(trip_start_timestamp) = '2017-05-01'
                    """

# Check your answer
q_2.check()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,pickup_community_area,trip_start_timestamp,trip_end_timestamp,trip_number
0,39.0,2017-05-01 01:15:00+00:00,2017-05-01 01:30:00+00:00,1
1,39.0,2017-05-01 02:15:00+00:00,2017-05-01 02:45:00+00:00,2
2,39.0,2017-05-01 05:15:00+00:00,2017-05-01 05:30:00+00:00,3
3,39.0,2017-05-01 05:45:00+00:00,2017-05-01 06:15:00+00:00,4
4,39.0,2017-05-01 06:45:00+00:00,2017-05-01 07:00:00+00:00,5


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [7]:
# Lines below will give you a hint or solution code
#q_2.hint()
#q_2.solution()

### 3) How much time elapses between trips?

The (partial) query in the code cell below shows, for each trip in the selected time frame, the corresponding `taxi_id`, `trip_start_timestamp`, and `trip_end_timestamp`. 

Your task in this exercise is to edit the query to include an additional `prev_break` column that shows the length of the break (in minutes) that the driver had before each trip started (this corresponds to the time between `trip_start_timestamp` of the current trip and `trip_end_timestamp` of the previous trip).  Partition the calculation by `taxi_id`, and order the results within each partition by `trip_start_timestamp`.

Some sample results are shown below, where all rows correspond to the same driver (or `taxi_id`).  Take the time now to make sure that the values in the `prev_break` column make sense to you!

![first_commands](https://i.imgur.com/qjvQzg8.png)

Note that the first trip of the day for each driver should have a value of **NaN** (not a number) in the `prev_break` column.

In [8]:
# Fill in the blanks below
break_time_query = """
                   SELECT taxi_id,
                       trip_start_timestamp,
                       trip_end_timestamp,
                       TIMESTAMP_DIFF(
                           trip_start_timestamp, 
                           LAG(trip_end_timestamp) 
                               OVER (
                                   PARTITION BY taxi_id 
                                   ORDER BY trip_start_timestamp
                               ),
                               MINUTE
                       ) as prev_break
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE DATE(trip_start_timestamp) = '2017-05-01' 
                   """

# Check your answer
q_3.check()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,taxi_id,trip_start_timestamp,trip_end_timestamp,prev_break
0,00e34f77f4495a6128d5b3312099c8f2f4ea25c2650e83...,2017-05-01 01:00:00+00:00,2017-05-01 01:30:00+00:00,NaN
1,00e34f77f4495a6128d5b3312099c8f2f4ea25c2650e83...,2017-05-01 02:15:00+00:00,2017-05-01 02:30:00+00:00,45.0
2,00e34f77f4495a6128d5b3312099c8f2f4ea25c2650e83...,2017-05-01 02:30:00+00:00,2017-05-01 02:45:00+00:00,0.0
3,1721f8d69b5f831e0c16c51510786ca43aaa367d23bec8...,2017-05-01 05:00:00+00:00,2017-05-01 05:15:00+00:00,NaN
4,1721f8d69b5f831e0c16c51510786ca43aaa367d23bec8...,2017-05-01 05:45:00+00:00,2017-05-01 06:15:00+00:00,30.0


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [9]:
# Lines below will give you a hint or solution code
#q_3.hint()
#q_3.solution()

# Keep going

Move on to learn how to query **[nested and repeated data](https://www.kaggle.com/alexisbcook/nested-and-repeated-data)**.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/advanced-sql/discussion) to chat with other learners.*